<a href="https://colab.research.google.com/github/Lalitha-DS/used-car-price-analysis/blob/main/used_car_price_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**a) Missing Values: Detect and Handle**




In [1]:
import pandas as pd

df = pd.read_csv('train.csv')

# Check missing values
missing_values = df.isnull().sum()
print("Missing values:\n", missing_values)


Missing values:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [2]:
def clean_mileage(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('kmpl', '').replace('km/kg', '').strip()
    try:
        return float(val)
    except:
        return None

df['Mileage'] = df['Mileage'].apply(clean_mileage)

# Impute numerical columns with median
df['Mileage'] = df['Mileage'].fillna(df['Mileage'].median())

def clean_engine(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('CC', '').strip()
    try:
        return float(val)
    except:
        return None

df['Engine'] = df['Engine'].apply(clean_engine)

df['Engine'] = df['Engine'].fillna(df['Engine'].median())

def clean_power(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('bhp', '').strip()
    try:
        return float(val)
    except:
        return None

df['Power'] = df['Power'].apply(clean_power)
df['Power'] = df['Power'].fillna(df['Power'].median())



# Drop rows where target 'Price' is missing
df = df.dropna(subset=['Price'])

# Drop rows where 'Seats' is missing (if very few)
df['Seats'] = df['Seats'].fillna(df['Seats'].mode()[0])  # mode = most common seat number





**Justification** :

Numerical columns are imputed with the median in order to reduce the influence that potential outliers may have on results. Target values and key features like Seats are either imputed or discarded depending on their importance and availability.


**b) Remove Units (Mileage, Engine, Power, New_Price)**


In [3]:
def extract_numeric(value, unit):
    if pd.isnull(value):
        return None
    return float(str(value).replace(unit, '').strip())

# Clean columns
df['Mileage'] = df['Mileage'].apply(lambda x: extract_numeric(x, 'kmpl') if 'kmpl' in str(x) else extract_numeric(x, 'km/kg'))
df['Engine'] = df['Engine'].apply(lambda x: extract_numeric(x, 'CC'))
df['Power'] = df['Power'].apply(lambda x: extract_numeric(x, 'bhp'))

# Clean New_Price if available
if 'New_Price' in df.columns:
    df['New_Price'] = df['New_Price'].str.replace(' Lakh', '')
    df['New_Price'] = pd.to_numeric(df['New_Price'], errors='coerce')


**c) One-Hot Encode 'Fuel_Type' and 'Transmission'**

In [4]:
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)



**d) Add a New Feature: Car Age**

In [5]:
from datetime import datetime
current_year = datetime.now().year

df['Car_Age'] = current_year - df['Year']


**e) Perform Select, Filter, Rename, Mutate, Arrange, Groupby + Summarize**

In [6]:
# Select relevant columns
selected_df = df[['Name', 'Location', 'Price', 'Mileage', 'Car_Age']]

# Filter: Cars less than 5 years old
filtered_df = df[df['Car_Age'] < 5]

# Rename column
df = df.rename(columns={'Kilometers_Driven': 'Odometer_km'})

# Mutate: Price per year
df['Price_per_Year'] = df['Price'] / df['Car_Age'].replace(0, 1)

# Arrange: Sort by Price descending
sorted_df = df.sort_values(by='Price', ascending=False)

# Groupby + Summarize
group_summary = df.groupby('Location')['Price'].agg(['mean', 'max', 'count']).reset_index()


In [7]:
df.to_csv("cleaned_used_cars.csv", index=False)
